# Gillespie Algorithm

```julia
import Pkg
Pkg.add(["Plots", "StatsBase"])
```

In [ ]:
#=
Stochastic chemical reaction: Gillespie Algorithm
Adapted from: Chemical and Biomedical Enginnering Calculations Using Python Ch.4-3
Reaction of A <-> B with rate constants k1 & k2
=#

using StatsBase, Random, Plots
Plots.gr(fmt=:png)

# Coded in the same manner as the Python counterpart
struct Gillespie
    propensityFuncs
    actionFuncs
    parameters
end

function (g::Gillespie)(u0, tend, tstart=zero(tend))
    t = tstart
    ts = [t]
    u = copy(u0)
    us = copy(transpose(u))
    p = g.parameters
    while t < tend
        # propensities of reactions
        ps = [f(u, p, t) for f in g.propensityFuncs]
        pTotal = sum(ps)
        dt = randexp() / pTotal
        f = sample(g.actionFuncs, Weights(ps))
        u = f(u, p, t)
        t += dt
        us = vcat(us, transpose(u))
        push!(ts, t) # Record t
    end
    return (ts = ts, us = us)
end

In [ ]:
parameters = (k1=1.0, k2=0.5)
propensityFuncs = [(u, p, t) -> p.k1 * u[1], (u, p, t) -> p.k2 * u[2]]
actionFuncs = [(u, p, t) -> u .+ [-1, 1] , (u, p, t) -> u .+ [1, -1]]
ssa = Gillespie(propensityFuncs, actionFuncs, parameters)

In [ ]:
ts, us = ssa([175, 25], 10.0)

In [ ]:
plot(ts, us, xlabel="time", ylabel="# of molecules", title = "SSA", label=["A" "B"])